In [7]:

import pandas as pd
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
import tweepy
import json
import os
from collections import Counter
import numpy as np
from collections import defaultdict
from matplotlib import pyplot as plt
import time
from datetime import datetime
from dateutil.parser import parse

In [24]:
# load tweets.json file
def load_json(file_name):
    result=[]
    with open(file_name, 'r') as fp:
        line=fp.readline()
        while line:
            tjson=json.loads(line) #decode json
            result.append({
                 "Date":tjson["created_at"],
                    "id":tjson["id_str"],
                 "user_name":tjson["user"]["name"],
                   
                    "favorite_count":tjson["favorite_count"], #number of favorites
                    "retweet_count":tjson["retweet_count"], #number of retweets
                    "user_id_str":tjson["user"]["id_str"],
                
                
            })

            line=fp.readline()
    return result


done


In [29]:
data_list=load_json("tweets.json")
df=pd.DataFrame(data=data_list)
print('done')
#df = df.drop_duplicates(subset=['id'], keep=False)

#converting date into more readable form 
Date_list= df["Date"].values
dates = []
for date in Date_list:
    dates.append(parse(date).date())
df["Date"]=dates
print(df)

done
             Date                   id             user_name  favorite_count  \
0      2020-04-22  1252984488667361285     ECO-HEALTHY YOUTH               0   
1      2020-04-22  1252984486159097858         edward fields               0   
2      2020-04-22  1252984485680824321       elizabeth judge               0   
3      2020-04-22  1252984484418420737       Scotch Partners               0   
4      2020-04-22  1252984473127354368           PLinNigeria               0   
...           ...                  ...                   ...             ...   
79767  2020-04-14  1250196716697128963        Karolyn Schalk               0   
79768  2020-04-14  1250196645532372992  Nick Reichert-Author               1   
79769  2020-04-14  1250196612409933829     Brendan Wissinger               0   
79770  2020-04-14  1250195677612634114               んひ丂刀ﾑ ♡               0   
79771  2020-04-14  1250195625376956416                   Al😷               0   

       retweet_count          user

In [31]:
#groupby date
group=df.groupby(by=["Date"])
new_df=group.aggregate({
    "id":["count"],
    "favorite_count":["sum"],
    "retweet_count":["sum"]
})
new_df.rename(columns={"id_str":"tweet_count"}, inplace=True)
print(new_df)
new_df.to_csv("climate_data.csv", index=True)


               id favorite_count retweet_count
            count            sum           sum
Date                                          
2020-04-14    261            302         78392
2020-04-15   7717          11854       1078744
2020-04-16   8173          12127        781434
2020-04-17   8652          13027        753467
2020-04-18   6719          17419        432779
2020-04-19   6771          10418        697365
2020-04-20   9687          15123       1150229
2020-04-21   9856          15031        788037
2020-04-22  21936          34818       2586017
